In [1]:
!date

Tue Apr 27 11:45:40 PDT 2021


# Install packages and download data

In [7]:
import sys
COLAB = "google.colab" in sys.modules


In [1]:
if COLAB:
    !git clone https://github.com/sbooeshaghi/BMGP_2020.git

Cloning into 'BMGP_2020'...
remote: Enumerating objects: 165, done.
remote: Counting objects: 100% (165/165), done.
remote: Compressing objects: 100% (134/134), done.
remote: Total 165 (delta 53), reused 121 (delta 23), pack-reused 0
Receiving objects: 100% (165/165), 59.24 MiB | 20.61 MiB/s, done.
Resolving deltas: 100% (53/53), done.


In [2]:
!pip --quiet install CITE-seq-Count==1.4.4

In [4]:
if COLAB:
    !pip install --quiet \
    kb-python==0.26.0 \
    anndata==0.7.6 \
    CITE-seq-Count==1.4.4

In [2]:
if COLAB:
    # download the relevant data
    !wget -O multiseq_fastqs.tar.gz \
    -q --show-progress --progress=bar:force  \
    https://caltech.box.com/shared/static/0scoe38xvcoxfd62xnm848pimh2zr1ip.gz \
    2>&1
    !tar -xf multiseq_fastqs.tar.gz

multiseq_fastqs.tar 100%[===================>]   6.47G  16.3MB/s    in 8m 1s   


# Preprocess with kb

In [3]:
%%time
!INDEX="BMGP_2020/references/multiseq/kite/features.idx" && \
F2B="BMGP_2020/references/multiseq/kite/f2b.txt" && \
FASTA="BMGP_2020/references/multiseq/kite/features.fa" && \
FB="BMGP_2020/references/multiseq/kite/feature_barcodes.txt" && \
kb ref \
-i $INDEX \
-g $F2B \
-f1 $FASTA \
--workflow kite \
$FB

[2021-05-18 11:04:19,470]    INFO Generating mismatch FASTA at BMGP_2020/references/multiseq/kite/features.fa
[2021-05-18 11:04:19,476]    INFO Creating transcript-to-gene mapping at BMGP_2020/references/multiseq/kite/f2b.txt
[2021-05-18 11:04:19,478]    INFO Skipping kallisto index because BMGP_2020/references/multiseq/kite/features.idx already exists. Use the --overwrite flag to overwrite.
CPU times: user 15.6 ms, sys: 17.8 ms, total: 33.4 ms
Wall time: 1.6 s


In [5]:
%%bash
FB="BMGP_2020/references/multiseq/kite/feature_barcodes.txt" && \
INDEX="BMGP_2020/references/multiseq/kite/features.idx" && \
F2B="BMGP_2020/references/multiseq/kite/f2b.txt" && \
FASTA="BMGP_2020/references/multiseq/kite/features.fa" && \
OUT="kite_out" && \
FASTQS=$(FASTQDIR="fastqs/" && paste -d" " \
<(ls $FASTQDIR | awk -v p=$FASTQDIR '{print p$0}' | grep 1) \
<(ls $FASTQDIR | awk -v p=$FASTQDIR '{print p$0}' | grep 2)) && \
kb count \
-i $INDEX \
-g $F2B \
-x 10xv2 \
-o $OUT \
--workflow kite \
--h5ad \
--filter bustools \
--overwrite \
$FASTQS

[2021-05-18 11:05:24,083]    INFO Using index BMGP_2020/references/multiseq/kite/features.idx to generate BUS file to kite_out from
[2021-05-18 11:05:24,084]    INFO         fastqs/SRR8890644_1.fastq.gz
[2021-05-18 11:05:24,084]    INFO         fastqs/SRR8890644_2.fastq.gz
[2021-05-18 11:08:29,745]    INFO Sorting BUS file kite_out/output.bus to kite_out/tmp/output.s.bus
[2021-05-18 11:08:39,947]    INFO Whitelist not provided
[2021-05-18 11:08:39,947]    INFO Copying pre-packaged 10XV2 whitelist to kite_out
[2021-05-18 11:08:40,275]    INFO Inspecting BUS file kite_out/tmp/output.s.bus
[2021-05-18 11:08:43,914]    INFO Correcting BUS records in kite_out/tmp/output.s.bus to kite_out/tmp/output.s.c.bus with whitelist kite_out/10xv2_whitelist.txt
[2021-05-18 11:08:48,255]    INFO Sorting BUS file kite_out/tmp/output.s.c.bus to kite_out/output.unfiltered.bus
[2021-05-18 11:08:57,092]    INFO Generating count matrix kite_out/counts_unfiltered/cells_x_features from BUS file kite_out/output.

# Preprocess with cite-seq-count

In [6]:
!gunzip BMGP_2020/references/whitelist_10xv2.txt.gz

In [ ]:
!R1="fastqs/SRR8890644_1.fastq.gz" && \
R2="fastqs/SRR8890644_2.fastq.gz" && \
FB="BMGP_2020/references/multiseq/cite_seq_barcodes.csv" && \
WL="BMGP_2020/references/whitelist_10xv2.txt" && \
/usr/bin/time -v CITE-seq-Count -R1 $R1  -R2 $R2 --no_umi_correction -t $FB -wl $WL -cbf 1 -cbl 16 -umif 17 -umil 26 -cells 65000 -o ./cite_seq_out

Loading whitelist
Counting number of reads
Started mapping
Processing 96,428,956 reads
CITE-seq-Count is running with 48 cores.
Processed 1,000,000 reads in 13.36 seconds. Total reads: 1,000,000 in child 12655
Processed 1,000,000 reads in 13.07 seconds. Total reads: 2,000,000 in child 12655
Mapping done for process 12655. Processed 2,008,936 reads
Processed 1,000,000 reads in 24.85 seconds. Total reads: 1,000,000 in child 12656
Processed 1,000,000 reads in 13.9 seconds. Total reads: 2,000,000 in child 12656
Mapping done for process 12656. Processed 2,008,936 reads
Processed 1,000,000 reads in 36.04 seconds. Total reads: 1,000,000 in child 12657
Processed 1,000,000 reads in 13.61 seconds. Total reads: 2,000,000 in child 12657
Mapping done for process 12657. Processed 2,008,936 reads
Processed 1,000,000 reads in 48.01 seconds. Total reads: 1,000,000 in child 12658
Processed 1,000,000 reads in 14.25 seconds. Total reads: 2,000,000 in child 12658
Mapping done for process 12658. Processed 2